In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAWW', 'AMCR', 'AVT', 'BCI', 'BHF', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'COUP', 'ESAB', 'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'LHCG', 'PEAK',
       'PFG', 'SBNY', 'SIVB', 'UA', 'UMPQ', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['AAWW', 'ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VAC', 'VICR', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW', 'XEL',
       'YUM'],
      dtype='object', length=213)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AAWW', 'ABNB', 'AIRC', 'ALGM', 'AM', 'ATCO', 'BHF', 'BJ', 'BRBR',
       'CAJ', 'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'COUP', 'CR',
       'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT', 'DTM', 'ESAB', 'ETH-USD', 'ETRN',
       'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO', 'IAA', 'IR', 'JBGS',
       'JHG', 'JJC', 'KD', 'LCID', 'LHCG', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT',
       'OGN', 'OTIS', 'PDD', 'PGNY', 'RCII', 'RFP', 'RIVN', 'SGOV', 'SHC',
       'SITM', 'SIVB', 'SJR', 'SWAV', 'UMPQ', 'VICI', 'VNT', 'VSCO', 'WH',
       'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 309
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 280


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      28
symbols with same volume:   213
symbols with no close:       68

symbols total before drop:                                        1,594
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   280

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,314    1,500
df Close (df_c) after dropped symbols:      1,314    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-04-10  137.380005  138.080002  135.860001  138.020004  1104600.0   
2023-04-11  138.869995  139.929993  138.149994  138.520004  1241100.0   
2023-04-12  140.190002  141.350006  137.820007  138.360001  1058400.0   
2023-04-13  139.179993  141.479996  138.990005  141.279999  1030900.0   
2023-04-14  140.360001  141.190002  138.929993  139.199997   917003.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-04-10  39.160000  40.150002  39.000000  39.779999  2452000.0  ...   
2023-04-11  40.520000  41.180000  40.150002  40.630001  4282000.0  ...   
2023-04-12  41.180000  41.250000  39.375000  39.639999  5256000.0  ...   
2023-04-13  40.380001  41.345001  40.119999  40.939999  4086500.0  ...   
2023-04-14  41.349998  41.430000  39.985001  40.400002  3341954.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-04-10  18.309999  18.870001  18.309999  18.830000  352800.0  20.030001   
2023-04-11  18.920000  19.670000  18.920000  19.490000  394000.0  20.580000   
2023-04-12  19.660000  19.670000  18.610001  18.709999  335500.0  20.980000   
2023-04-13  18.870001  18.870001  18.320000  18.370001  266200.0  20.510000   
2023-04-14  18.520000  19.179001  18.450001  18.600000  231135.0  20.680000   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-04-10  20.629999  20.030001  20.469999  1157700.0  
2023-04-11  21.020000  20.510000  20.809999  1224700.0  
2023-04-12  21.139999  20.559999  20.580000   637700.0  
2023-04-13  21.020000  20.360001  20.910000   731900.0  
2023-04-14  21.150000  20.445000  20.590000   845111.0  

[5 rows x 6570 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-04-10,138.020004,39.779999,14.15,118.901749,162.029999,34.209999,33.150002,159.809845,166.380005,43.910000,...,29.657358,128.149994,301.959991,77.330002,44.070000,29.870001,28.690001,170.869995,18.830000,20.469999
2023-04-11,138.520004,40.630001,14.32,120.669617,160.800003,35.509998,33.869999,159.908936,166.550003,43.930000,...,28.940001,131.380005,304.029999,78.169998,43.810001,30.110001,28.690001,171.210007,19.490000,20.809999
2023-04-12,138.360001,39.639999,13.00,119.839996,160.100006,34.200001,34.450001,160.880005,165.419998,43.689999,...,28.950001,131.589996,301.410004,76.919998,44.750000,30.290001,28.230000,171.449997,18.709999,20.580000
2023-04-13,141.279999,40.939999,12.95,123.250000,165.559998,35.029999,34.840000,162.690002,167.380005,44.189999,...,29.700001,132.520004,306.170013,77.879997,44.930000,31.150000,28.510000,173.539993,18.370001,20.910000
2023-04-14,139.199997,40.400002,12.79,123.599998,165.210007,36.470001,35.139999,161.589996,167.130005,43.189999,...,29.500000,132.369995,303.079987,76.970001,44.680000,29.950001,28.410000,174.600006,18.600000,20.590000
